Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

---

In [ ]:
using LsqFit
using DifferentialEquations
using Plots
using Base.Test

# IL027 Computer Modelling for All

## Lecture 2- Assignment

Topics covered:

 - Ordinary differential equations
 - Plots and visualisation
 - Root finding 
 
Before attempting the questions,

* log in to JuliaBox
* ensure you have made no changes to the notebooks in the `ComputerModelling` repository, if so make a copy of those notebooks in your home space.
* update the `ComputerModelling` repository by clicking on `Git` button from JuliaBox homepage, then pressing `Synchronize from remote` (double arrow icon).
* make a copy of the `L2-Assignment.ipynb` notebook

**WARNING:** You should never edit the cloned notebook directly, but always edit a copy in your home folder. If you edit the cloned copy them it will be overwritten, next time you pull the latest update from the IL027 repository.

----

### Introduction to World Population Modelling

The United Nations considers [three possible scenarios](https://en.wikipedia.org/wiki/Projections_of_population_growth) of population growth in this century: (1) Exponential Growth, (2) Capped Growth and (3) Declining Population  The growth in each scenario can be forecasted by different variants of a ordinary differential equation [population model](https://en.wikipedia.org/wiki/Logistic_function#In_ecology:_modeling_population_growth)  defined by

\begin{align*}
    \frac{dP}{dt} = \alpha\, P\, \left(1-\frac{P}{K}\right)
\end{align*}

where $P$ is the population in billions of people, $t$ is the time in years, $\alpha$ is the population growth rate and $K$ is the carrying capacity in billions of people.

We begin by creating vectors with the world's population data for the period 1900 to 2017 [(data source)](https://esa.un.org/unpd/wpp/Download/Standard/Population/).

In [ ]:
# historic population data from 1900 to 2017, in billions of people
population = [1.65, 2.52, 5.98, 6.70, 6.90, 7.05, 7.48]
year = [1900, 1950, 1999, 2008, 2010, 2012, 2017];

**Question 1. (20%)** Plot a graph of population as a function of year using `Plots.jl`. Your plot should have axis labels and markers but no lines joining the data points - see the documentation on `scatter()` for suggestions.

In [ ]:
YOUR CODE HERE

### Scenario 1. - Exponential Growth

The Exponential Growth scenario assumes there are so many resources on Earth that the carrying capacity is infinite, $K \to \infty$, giving

\begin{align*}
    \frac{dP}{dt} = \alpha P
\end{align*}

The population growth rate, $\alpha$, is unknown, but we can estimate it via a least squares fit to the population data from 1900 to 2017.

In [ ]:
P0 = population[1] # world population in 1900
model(x, p) = P0*exp.(p[1].*x)
fit = curve_fit(model, year .- 1900, population, [0.01])
α = fit.param[1] # estimated population growth rate

Go through the above code and understand how it works. Try changing the initial guess $\alpha=0.01$ to see how accurate it needs to be.

**Question 2a. (10%)** Define a function `dP1(t, P)` for the exponential growth population model. Use `ODEProblem()` and `solve()` from `DifferntialEquations.jl` to integrate this ODE to solve the model between 1900 and 2100. Name your solution `sol1`. 

In [ ]:
YOUR CODE HERE

In [ ]:
# use this cell to test your answer
@test sol1(1900) == 1.65
@test isapprox(sol1(2100), 21.919171907536615, atol=1e-3)

**Question 2b (10%).** Plot the results in comparison with the historic data between 1900 and 2017.

In [ ]:
YOUR CODE HERE

The model predicts that there will be nearly 22 billion people in the year 2100! The predicted population is very high because the model only considers continual growth. Note that between 1900 and 2017 we are **interpolating** and the fit is good, while after 2017 we are **extrapolating** so we are less confident.

### Scenario 2. Capped Growth

Since resources available on Earth are finite, the world's population in reality would not sustain exponential growth indefinitely. The following variant of the model takes this into account, and assumes the carrying capacity (denoted by $K$) is an upper population threshold (i.e. population cannot grow bigger than it). The ODE is:

\begin{align*}
    \frac{dP}{dt} = \alpha P \left(1-\frac{P}{K} \right)
\end{align*}

where $P$ is the population in billions of people, $t$ is the time in years, $\alpha$ is the population growth rate, and $K$ is the carrying capacity in billions of people. 

**Question 3a (10%).** Predict the world's population in the period 2017-2100 using this model. Use the value of $\alpha$ found in last section and take $K$ to be equal to 11 billion people. Note that the initial condition should be the 2017 population since we are only imposing the gap on growth for the future. Name your new solution `sol2`.

In [ ]:
K = 11 # population cap in billions of people (carrying capacity)
YOUR CODE HERE

In [ ]:
# use this cell to test your answer
@test sol2(2017.) ≈ 7.48
@test isapprox(sol2(2100), 9.475714071145484, atol=1e-3)

**Question 3b (10%).** Produce a plot comparing the exponential growth and capped growth scenarios - you should adjust the x- and y-limits of your plot using e.g. the `xlims!()` and `ylims!()` functions to give a well presented plot.

In [ ]:
YOUR CODE HERE

The predicted population for the year 2100 is around 9.5 billion people, and in future centuries it will end up converging to the carrying capacity of 11 billion.

### Scenario 3. Declining Population

In this scenario, the population growth rate $\alpha(t)$ does not remain constant, but rather decreases as a consequence of overpopulation and resource scarcity. This leads the world's population to decrease after reaching a maximum point. The ODE is:

\begin{align*}
    \frac{dP}{dt} = \alpha(t)\,P \left(1 - \frac{P}{K} \right)
\end{align*}

Note that now $\alpha$ depends on time. 

**Question 4. (10%)** Define a function `alpha_lin(t)` that returns $\alpha(t)$ assuming the rate diminishes by a fixed amount of `dα = 0.001` every year after 2017.

In [ ]:
α0 = α    # initial growth rate
dα = 0.001 # change of growth rate per year
YOUR CODE HERE

In [ ]:
@test alpha_lin(2017) ≈ α
@test alpha_lin(2100) ≈ -0.0700670533654165

Now we are ready to predict the world's population under the Declining Population scenario.

**Question 5a. (10%)** Solve the ODE using the $\alpha(t)$ that you just defined for the period 2017-2100. Name your solution `sol3`.

In [ ]:
YOUR CODE HERE

In [ ]:
#use this cell to test your answer
@test isapprox(sol3(2100), 1.8212687312577343, atol=1e-3)

**Question 5b. (10%)** Produce a final plot comparing all three scenarios.

In [ ]:
YOUR CODE HERE

**Question 6. (10%)** Write a function `pop5bn(sol3)` which returns the year during which the population falls below 5 billion according to the solution `sol3`. Note that the year returned should be an integer. Look up the documentation for `round()` and `convert()` to see how to round to the nearest whole number and how to convert from float to int.

In [ ]:
YOUR CODE HERE

In [ ]:
# Use this cell to test your answer
@test typeof(pop5bn(sol3)) == Int64
@test sol3(pop5bn(sol3)) > 5.0
@test sol3(pop5bn(sol3) + 1) < 5.0

### Bonus Question (no marks)

Write a function `min_change_in_growth_rate(year, target_pop)` that returns the minimum yearly change in $\alpha$ so that the population is less than `pop` billion in `year`. Use it to find the minimum yearly change in growth rate required for the population to fall to 5 billion in 2100.

*Hint:* First write a function `pop(dα, year)` that sets up and solves an ODE to return the population in `year` as a function of the yearly change in $\alpha$ `dα`, then use `fzeros()` to find the value of `dα` for which `pop(dα, year)` is equal to `target_pop`

In [ ]:
YOUR CODE HERE

In [ ]:
# Use this cell to test your code
@test min_change_in_growth_rate(2100., 5.0) ≈ 0.000583404709651563
@test pop(min_change_in_growth_rate(2100., 5.0), 2100.) ≈ 5.0

The minimum annual decrease in growth rate for the population to fall to 5 billion in 2100 is approximately 0.000583.